In [1]:
from matplotlib import pyplot as plt
from tqdm import tqdm
import SimpleITK as sitk
import glob
import numpy as np
from utils import load_itk, resample, get_segmented_lungs

In [2]:
INPUT_FOLDER = r'/data/MSD/Task06_Lung/imagesTr/lung*'
file_list = glob.glob(INPUT_FOLDER)
print(len(file_list))
file_list.sort()

63


In [3]:
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import os
import matplotlib

for file in tqdm(file_list):
    img, origin, spacing = load_itk(file)
    img2, spacing2 = resample(img, spacing)
    img3 = np.asarray([get_segmented_lungs(im) for im in img2])

    raw_name = file.split('/')[5].split('.')[0]
    
    ## visualize generated masks

    if not os.path.exists('/data/MSD-lung-segment-preview/' + raw_name + '.png'):
        p = img3.transpose(2, 1, 0)
        verts, faces, normals, values = measure.marching_cubes(p, 0)

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        mesh = Poly3DCollection(verts[faces], alpha=0.20)
        face_color = [0.45, 0.45, 0.75]
        mesh.set_facecolor(face_color)
        ax.add_collection3d(mesh)

        ax.set_xlim(0, p.shape[0])
        ax.set_ylim(0, p.shape[1])
        ax.set_zlim(0, p.shape[2])

        plt.savefig('/data/MSD-lung-segment-preview/' + raw_name + '.png')
        plt.close()
        matplotlib.use('Agg')

    ### saving masks
    current_name = raw_name + '.nrrd'
    if not os.path.exists(current_name):
        ma = sitk.GetImageFromArray(img3)
        sitk.WriteImage(
            ma,
            '/data/MSD-lung-segment/' + current_name, True
        )

  0%|          | 0/63 [00:00<?, ?it/s]/workspace/masters-object-detection/LungSegmentationValidation/utils.py:116: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  l = convex_hull_image(l)
  8%|▊         | 5/63 [10:33<2:10:11, 134.68s/it]/workspace/masters-object-detection/LungSegmentationValidation/utils.py:115: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  r = convex_hull_image(r)
100%|██████████| 63/63 [2:01:03<00:00, 115.29s/it]  
